In [1]:
from graph_ds import PyH3Graph
from pois_to_h3 import get_pois_h3
from ghsl_processing import get_origins, city_boundaries_to_h3 

import pandas as pd
import h3.api.numpy_int as h3
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import seaborn as sns
import pickle


def plot_hex_df(df, markersize, color_column=None, color_categorical=False):
    df[['y','x']] = list(df['h3_index'].apply(h3.h3_to_geo))
    # Convert the pandas DataFrame to a GeoPandas DataFrame with a Point geometry column
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y))

    fig, ax = plt.subplots(figsize=(10,10))
    # Show the GeoPandas DataFrame

    if color_column:
        gdf.plot(ax=ax, column=color_column, categorical=color_categorical,
                  legend=True, markersize=markersize, alpha=0.5, cmap='Reds',
                  vmin=0, vmax=25)
    else:
        gdf.plot(ax=ax, markersize=markersize, alpha=0.5)

    #plt.xlim(12.391681,12.737388)
    #plt.ylim(55.549206, 55.759991)

    cx.add_basemap(ax = ax, crs="EPSG:4326")
    #plt.show()

def routing(category_set, origins, destinations, num_origins, graph):
    mins={}
    category_travel_time_means = {}
    for c in category_set:
        cat_destinations = list(destinations[destinations['category']==c]['h3_index'])
        ds = graph.matrix_distance(origins=origins, destinations=cat_destinations, dynamic_infinity=True)

        # they might be different lengths, so we can't use a df
        mins[c]={k:np.nanmin(np.array(v,dtype=np.float64)) for k,v in ds.items() if v}
        category_travel_time_means[c] = np.nanmean(list(mins[c].values()))

    return category_travel_time_means, mins


/Users/petermehler/opt/anaconda3/envs/graphs/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# # denver, cl
# osm_file = "../resources/denver-processed.osm.pbf"
# gtfs_files = ["../resources/denver_gtfs.zip"]
# ghsl_file = "../resources/GHS_BUILT_C_MSZ_E2018_GLOBE_R2022A_54009_10_V1_0_R5_C10.tif"

# city_name = "Denver, Colorado"

# copehagen, dk
osm_file = "../resources/osm/København Kommune_processed.osm.pbf"
gtfs_files = ["../resources/rejseplanen_GTFS.zip"]
ghsl_file = '../resources/GHS_BUILT_C_MSZ_E2018_GLOBE_R2022A_54009_10_V1_0_R3_C19_Denmark.tif'

city_name = "København Kommune"

In [3]:
# Parameters

# 12 is around 10 meter resolution
H3_RES = 12

# how many origins to sample
num_origins = 1000

#essential filter will be the least restrictive
essential_filter = {
    "amenity":["pharmacy","dentist","clinic","doctors","school"],
    "shop":["supermarket","greengrocer","medical_supply","grocery","wholesale"],
    "healthcare":["clinic","doctor","pharmacy","dentist"],
    "leisure":["park"], "sport":True
}

# categories considered for the n-minute calculation
category_set = ['pharmacy','park','supermarket','sport', 'school']

# how to map osm tags to categories
osm_tag_mapping = {
    "medical_supply":"pharmacy",
    "greengrocer":"supermarket",
    "wholesale":"supermarket",
    "grocery":"supermarket",
    "clinic":"doctor",
    "doctors":"doctor",
    'pitch':'sport',
    'track':'sport',
    'sports_centre':'sport'
}

In [21]:
results = {}

city_bounds_h3, bbox, bbox_pois = city_boundaries_to_h3([city_name])

origins = get_origins(H3_RES, [city_name], bbox, ghsl_file, city_bounds_h3)
print("origins", len(origins))

origins_sample = list(origins[origins['residential_bool']==1].sample(num_origins, replace=True)['h3_index'])


destinations = get_pois_h3(osm_file, essential_filter, H3_RES, category_set, osm_tag_mapping, [city_name])

for g_type in ['all', 'walk', 'walk+bike', 'walk+transit']:
        print(g_type)
        # build the graph
        graph = PyH3Graph(bike_penalty=1, k_ring=2, layers=g_type)
        graph.create(osm_path=osm_file,gtfs_paths=gtfs_files)
        # do routing
        category_means, route_mins = routing(category_set, origins_sample, destinations, num_origins, graph)
        # save results as dictionary
        results.update({g_type:{'category_means':category_means, 'route_mins':route_mins}})

file already exists for København Kommune
origins 163542
file already exists for København Kommune
all
processing osm pbf file: ../resources/processed/København Kommune_processed.osm.pbf
converted OSM file into 382287 edges
osm graph created with 326898 nodes in 2.2387033 s
getting GTFS feed from ../resources/rejseplanen.zip
routes: 1631
gtfs graph created with 103525 nodes in 9.606469 s
merged gtfs graph into osm graph, now has 428651 nodes, took 90 ms
hash: 4736217103143947563


/tmp/ipykernel_209231/4198998382.py:44: RuntimeWarning: All-NaN slice encountered
  mins[c]={k:np.nanmin(np.array(v,dtype=np.float64)) for k,v in ds.items() if v}


walk
processing osm pbf file: ../resources/processed/København Kommune_processed.osm.pbf
converted OSM file into 328446 edges
osm graph created with 274679 nodes in 1.7559271 s
hash: 14700497474873797144
walk+bike
processing osm pbf file: ../resources/processed/København Kommune_processed.osm.pbf
converted OSM file into 382287 edges
osm graph created with 326898 nodes in 1.9905901 s
hash: 18360007200651104482
walk+transit
processing osm pbf file: ../resources/processed/København Kommune_processed.osm.pbf
converted OSM file into 328446 edges
osm graph created with 274679 nodes in 1.7512894 s
getting GTFS feed from ../resources/rejseplanen.zip
routes: 1631
gtfs graph created with 103525 nodes in 9.47268 s
merged gtfs graph into osm graph, now has 376432 nodes, took 87 ms
hash: 11603647893462833771


In [54]:
# From 1 point to all others

# copehnagen hovedbanegard
hbg = {'x': 55.674599, 'y': 12.564449}
norreport = {'x': 55.683588, 'y': 12.571663}
origin = hbg
#convert point to h3
origin_h3 = [h3.geo_to_h3(origin['x'], origin['y'], H3_RES)]

results = {}

city_bounds_h3, bbox, bbox_pois = city_boundaries_to_h3([city_name])
origins_sample = origin_h3
destinations = get_pois_h3(osm_file, essential_filter, H3_RES, category_set, osm_tag_mapping, [city_name])
destinations = destinations['h3_index'].values

file already exists for København Kommune


In [60]:
weight_options = {'bike_penalty': 1.0,
                            'wait_time_multiplier': 1,
                            'walk_speed': 1.4,
                            'bike_speed': 4.5}

for g_type in ['all', 'walk+transit']:
        print(g_type)
        # build the graph
        graph = PyH3Graph(weight_options=weight_options, k_ring=5, layers=g_type)
        graph.create(osm_path=osm_file,gtfs_paths=gtfs_files)
        # do routing
        ds = graph.matrix_distance(origins=destinations, destinations=origins_sample, dynamic_infinity=False)

walk+transit
processing osm pbf file: ../resources/osm/København Kommune_processed.osm.pbf
converted OSM file into 328446 edges
osm graph created with 274679 nodes in 14.547345 s
getting GTFS feed from ../resources/rejseplanen_GTFS.zip
routes: 1690
gtfs graph created with 104882 nodes in 40.74683 s
merged gtfs graph into osm graph, now has 377791 nodes, took 509 ms
hash: 11956091092678676292


TypeError: argument 'origins': 'str' object cannot be interpreted as an integer

In [59]:
pd.DataFrame(ds)

,631049684383439359
0,19.963535
1,35.982066
2,24.262200
3,20.536982
4,35.386302
...,...
39348,16.597331
39349,15.344635
39350,20.369417
39351,27.504112


In [ ]:
def kring_smoothing(df, hex_col, metric_col, k):
    df['d'] = df.d2-df.d1
    dfk = df[[hex_col]] 
    dfk.index = dfk[hex_col]
    dfs =  (dfk[hex_col]
                 .apply(lambda x: pd.Series(list(h3.k_ring(x,k)))).stack()
                 .to_frame('hexk').reset_index(1, drop=True).reset_index()
                 .merge(df[[hex_col,metric_col]]).fillna(0)
                 .groupby(['hexk'])[[metric_col]].sum().divide((1 + 3 * k * (k + 1)))
                 .reset_index()
                 .rename(index=str, columns={"hexk": hex_col}))
    dfs['lat_x'] = dfs[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
    dfs['lon_x'] = dfs[hex_col].apply(lambda x: h3.h3_to_geo(x)[1]) 
    return dfs

In [57]:
ds.keys()

dict_keys([631049684383439359])

In [22]:
import polars as pl
from lets_plot import *
LetsPlot.setup_html()

In [23]:
result_data = []
for g_type, result in results.items():
    route_mins = result['route_mins']
    for category, values in route_mins.items():
        for h3_index, value in values.items():
            lat, lon = h3.h3_to_geo(h3_index)
            result_data.append([g_type, category, h3_index, lat, lon, value])

result_df = pl.DataFrame(result_data, columns=['graph_type', 'category', 'h3', 'lat', 'lon', 'travel_time'])

result_df.head()

/tmp/ipykernel_209231/865589535.py:9: DeprecationWarning: `columns` is deprecated as an argument to `__init__`; use `schema` instead.
  result_df = pl.DataFrame(result_data, columns=['graph_type', 'category', 'h3', 'lat', 'lon', 'travel_time'])


graph_type,category,h3,lat,lon,travel_time
str,str,i64,f64,f64,f64
"""all""","""pharmacy""",631049690226203647,55.663315,12.481003,6.795071
"""all""","""pharmacy""",631049683897030143,55.640385,12.530008,21.082284
"""all""","""pharmacy""",631049688307779583,55.712998,12.548218,9.318954
"""all""","""pharmacy""",631049684194895359,55.712052,12.568221,8.930664
"""all""","""pharmacy""",631049690219572223,55.655612,12.490519,8.971958


In [24]:
plot_df = result_df.filter((pl.col('travel_time').is_not_nan()) & (pl.col('travel_time') < 45.0))

In [25]:
ggplot(plot_df) + \
    geom_livemap() + \
    geom_point(aes(x='lon', y='lat'), size=0.5) + \
    ggtitle(f"Origins in {city_name}")

In [26]:
ggplot(plot_df) + \
    geom_point(aes(x='lon', y='lat', color='travel_time'), alpha=0.5) + \
    scale_color_gradient(low='#f2f0f7', high='#54278f') + \
    facet_wrap(["graph_type", "category"], ncol=5) + \
    coord_fixed() + \
    ggtitle(f"Minimum travel times to essential services in [{city_name}]") + \
    ggsize(1000, 700)

In [27]:
ggplot(plot_df) + \
    geom_density(aes(x='travel_time', fill='category'), alpha=0.3) + \
    xlim(0, 30) + \
    facet_wrap(["graph_type"], ncol=5) + \
    ggtitle(f"Minimum travel time to essential services in [{city_name}], grouped by network type")

In [28]:
benefit_df = plot_df.groupby(['h3', 'category']).agg(
    pl.col('travel_time'),
    pl.col('travel_time').len().alias('count'),
).filter(pl.col('count') == 4).groupby(['h3', 'category']).agg(
    (pl.col('travel_time').flatten().take(1) - pl.col('travel_time').flatten().take(2)).alias('bike_benefit'),
    (pl.col('travel_time').flatten().take(1) - pl.col('travel_time').flatten().take(3)).alias('transit_benefit'),
    (pl.col('travel_time').flatten().take(1) - pl.col('travel_time').flatten().take(0)).alias('multimodal_benefit'),
)

In [29]:
ggplot(benefit_df) + \
    geom_histogram(aes(x='multimodal_benefit', fill='category'), alpha=0.5, bins=40) + \
    xlim(0, 15) + \
    ggtitle(f"Multimodal travel time benefit over walking in [{city_name}]")

In [30]:
benefit_df.head()

h3,category,bike_benefit,transit_benefit,multimodal_benefit
i64,str,f64,f64,f64
631049684136901631,"""park""",0.0,0.0,0.0
631049683612232703,"""park""",0.0,0.0,0.0
631049683905285119,"""park""",0.0,0.0,0.0
631049683595779071,"""park""",0.0,0.0,0.0
631049690730740735,"""park""",0.0,0.0,0.0


In [31]:
benefit_df.groupby("category").agg(
    pl.col('bike_benefit').mean().alias('bike_benefit'),    
    pl.col('transit_benefit').mean().alias('transit_benefit'),
    pl.col('multimodal_benefit').mean().alias('multimodal_benefit'),
)

category,bike_benefit,transit_benefit,multimodal_benefit
str,f64,f64,f64
"""supermarket""",0.708869,0.172376,0.718287
"""sport""",2.111979,0.863301,2.189791
"""park""",0.205309,0.089367,0.256294
"""school""",0.817156,0.172926,0.826814
"""pharmacy""",2.570547,1.387064,2.860314


In [36]:
plot_df.head()

graph_type,category,h3,lat,lon,travel_time
str,str,i64,f64,f64,f64
"""all""","""pharmacy""",631049690226203647,55.663315,12.481003,6.795071
"""all""","""pharmacy""",631049683897030143,55.640385,12.530008,21.082284
"""all""","""pharmacy""",631049688307779583,55.712998,12.548218,9.318954
"""all""","""pharmacy""",631049684194895359,55.712052,12.568221,8.930664
"""all""","""pharmacy""",631049690219572223,55.655612,12.490519,8.971958


In [38]:
plot_df.groupby(["graph_type", "h3"]).agg(
    pl.col('travel_time').max().alias('max_travel_time')
).groupby("graph_type").agg(
    pl.col('max_travel_time').mean().alias('mean_max_travel_time')
)

graph_type,mean_max_travel_time
str,f64
"""walk+transit""",12.676367
"""walk""",14.543637
"""walk+bike""",11.198887
"""all""",10.823698
